In [1]:
import os
import pandas as pd
from glob import glob
root = r'E:\20230527-LiXueJie\patches'

patches = pd.DataFrame(glob(os.path.join(root, '*', '*.jpg')), columns=['fpath'])
patches['ID'] = patches['fpath'].map(lambda x: os.path.basename(os.path.dirname(x)))
patches

,fpath,ID
0,E:\20230527-LiXueJie\patches\16-00986-19-20\16...,16-00986-19-20
1,E:\20230527-LiXueJie\patches\16-00986-19-20\16...,16-00986-19-20
2,E:\20230527-LiXueJie\patches\16-00986-19-20\16...,16-00986-19-20
3,E:\20230527-LiXueJie\patches\16-00986-19-20\16...,16-00986-19-20
4,E:\20230527-LiXueJie\patches\16-00986-19-20\16...,16-00986-19-20
...,...,...
2636487,E:\20230527-LiXueJie\patches\H20-00569\H20-005...,H20-00569
2636488,E:\20230527-LiXueJie\patches\H20-00569\H20-005...,H20-00569
2636489,E:\20230527-LiXueJie\patches\H20-00569\H20-005...,H20-00569
2636490,E:\20230527-LiXueJie\patches\H20-00569\H20-005...,H20-00569


In [2]:
import pandas as pd
import os
from onekey_algo.custom.components.comp2 import split_dataset4sol

root = r'D:\20230527-LiXueJie'
data = pd.read_csv(os.path.join(root, 'label.csv'))
data

,ID,label
0,16-00986-19-20,1
1,18-14086-19-20,0
2,18-16868-25-26,1
3,18-21981-24,0
4,19-03150-23-24,0
...,...,...
147,20-20669-35-36,0
148,20-20965-25-26,0
149,20-21026-7-8,1
150,20-21033-33-34,0


In [3]:
label_data = data[['ID', 'label']]
print(label_data['label'].value_counts())
label_data

1    76
0    76
Name: label, dtype: int64


,ID,label
0,16-00986-19-20,1
1,18-14086-19-20,0
2,18-16868-25-26,1
3,18-21981-24,0
4,19-03150-23-24,0
...,...,...
147,20-20669-35-36,0
148,20-20965-25-26,0
149,20-21026-7-8,1
150,20-21033-33-34,0


In [4]:
subsets = split_dataset4sol(label_data, label_data['label'], cv=False, n_trails=5, save_dir=root, shuffle=True, test_size=0.3)

# 弱监督学习

In [5]:
for idx, (train, test) in enumerate(subsets):
    train = pd.merge(train, patches, on='ID', how='inner')
    train[['fpath', 'label']].to_csv(os.path.join(root, 'split_info', f'semi_train-RND-{idx}.txt'), index=False, header=False, sep='\t')
    test = pd.merge(test, patches, on='ID', how='inner')
    test[['fpath', 'label']].to_csv(os.path.join(root, 'split_info', f'semi_val-RND-{idx}.txt'), index=False, header=False, sep='\t')

# 全监督学习

In [16]:
patch_info = pd.read_csv(os.path.join(root.replace('D', 'E'), 'patches', 'patch_info.csv'))
patch_info['ID'] = patch_info['filename'].map(lambda x: os.path.dirname(x))
patch_info['label_id'] = patch_info['label_id'].map(lambda x: x if x in [0, 1] else 1)
patch_info

,filename,label_id,label,info,ID
0,16-00986-19-20\16-00986-19-20_80192_155904_0_0...,0,_background_,"{""_background_"": 1.0}",16-00986-19-20
1,16-00986-19-20\16-00986-19-20_80192_155904_0_4...,0,_background_,"{""_background_"": 1.0}",16-00986-19-20
2,16-00986-19-20\16-00986-19-20_80192_155904_0_8...,0,_background_,"{""_background_"": 1.0}",16-00986-19-20
3,16-00986-19-20\16-00986-19-20_80192_155904_0_1...,0,_background_,"{""_background_"": 1.0}",16-00986-19-20
4,16-00986-19-20\16-00986-19-20_80192_155904_0_1...,0,_background_,"{""_background_"": 1.0}",16-00986-19-20
...,...,...,...,...,...
9800358,H20-00569\H20-00569_80192_175168_79744_172928_...,0,_background_,"{""_background_"": 1.0}",H20-00569
9800359,H20-00569\H20-00569_80192_175168_79744_173376_...,0,_background_,"{""_background_"": 1.0}",H20-00569
9800360,H20-00569\H20-00569_80192_175168_79744_173824_...,0,_background_,"{""_background_"": 1.0}",H20-00569
9800361,H20-00569\H20-00569_80192_175168_79744_174272_...,0,_background_,"{""_background_"": 1.0}",H20-00569


In [17]:
patch_info['fpath'] = patch_info['filename'].map(lambda x: os.path.join(root.replace('D', 'E'), 'patches', x))
patch_info = pd.merge(patches[['fpath']], patch_info, on='fpath', how='inner')
for idx, (train, test) in enumerate(subsets):
    train = pd.merge(train, patch_info, on='ID', how='inner')
    train[['fpath', 'label_id']].to_csv(os.path.join(root, 'split_info', f'train-RND-{idx}.txt'), index=False, header=False, sep='\t')
    test = pd.merge(test, patch_info, on='ID', how='inner')
    test[['fpath', 'label_id']].to_csv(os.path.join(root, 'split_info', f'val-RND-{idx}.txt'), index=False, header=False, sep='\t')

# 仅保存ROI区域

In [1]:
import os
import pandas as pd

root = r'E:/20230527-LiXueJie/models/inception_v3/viz/'

for f in os.listdir(root):
    if f.startswith('BST_'):
        d = pd.read_csv(os.path.join(root, f), sep='\t', names=['fname', 'prob', 'pred', 'gt'])
        d[d['gt'] == 1].to_csv(os.path.join(root, f.replace('.txt', '_roi.txt')), sep='\t', index=False, header=False)